# Script for lung segmentation and lobar fractional iodine calculation (batch processing)

Accompanying Jupyter notebook for the research article: 

Strotzer QD, Heidemanns S, Mayr V, Stuerzl R, Meiler S, Schmidt D, Blaas S, Grosse J, Hellwig D, Stroszczynski C, Hamer OW (2023) Head-to-Head Comparison of Dual-Source and Split-Beam Filter Multi-Energy CT versus SPECT/CT for Assessing Lobar Lung Perfusion in Emphysema. Radiology: Cardiothoracic Imaging. https://doi.org/10.1148/ryct.220273 

Provided data is structured as described below, DICOM slices are loaded, lobar segmentation is applied, and lobar volume and perfusion fraction are calculated for each case stored in cases.txt. Results are saved in tabular format as *results.xlsx*.

## Prerequisites:
- SimpleITK
- numpy
- pandas
- lungmask
- os

## Args: 
- cases.txt: a text file containing case ids (one per line)
- for each case, a folder containing a structural chest CT (one DICOM file per slice) 
and an equally spaced dual-energy-CT-derived iodine map (e.g., exported from syngo.via CT Dual Energy)

## Output:
- caseID_segmentation.nii.gz
- caseID_backdrop.nii.gz
- caseID_iodine_map.nii.gz
- results.xlsx

## Data Structure: 

```bash
main_directory/
├── caseID_01/
│   ├── backdrop/ # Contains DICOM-files (one per slice)     
│   └── iodine_map/ # Contains DICOM-files (one per slice)   
├── caseID_02/
│   ├── ...      
...    
└── cases.txt     
```

## License:
```python
__author__ = "Quirin D. Strotzer"
__license__ = "Apache License 2.0"
__version__ = "1.0"
__email__ = "quirin.strotzer@ukr.de"
```

## Cite as:
Strotzer QD, Heidemanns S, Mayr V, Stuerzl R, Meiler S, Schmidt D, Blaas S, Grosse J, Hellwig D, Stroszczynski C, Hamer OW (2023) Head-to-Head Comparison of Dual-Source and Split-Beam Filter Multi-Energy CT versus SPECT/CT for Assessing Lobar Lung Perfusion in Emphysema. Radiology: Cardiothoracic Imaging. https://doi.org/10.1148/ryct.220273 

## Prerequisites

In [1]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
from lungmask import LMInferer
import os

main_directory = '/path/to/main/directory'

# Load list of cases
cases = [line.rstrip('\n') for line in open(os.path.join(main_directory, 'cases.txt'))]

# Names and values for lobes resulting from lungmask segmentation algorithm
lobes = [('left_upper', 1), ('left_lower', 2), ('right_upper', 3),
    ('middle', 4), ('right_lower', 5)]

In [2]:
def load(in_path, out_path, name):

    '''
    loads DICOM slices from in_path, saves compressed nifti under
    out_path/name.nii.gz, and returns itk image
    '''
    
    print('loading image:', in_path)
    reader = sitk.ImageSeriesReader()
    reader.LoadPrivateTagsOn()
    filenames_DICOM = reader.GetGDCMSeriesFileNames(in_path)
    reader.SetFileNames(filenames_DICOM)
    loaded_image = reader.Execute()
    loaded_image.SetOrigin((0, 0, 0))
    sitk.WriteImage(loaded_image, os.path.join(out_path, f'{name}.nii.gz'))
    
    return loaded_image

In [3]:
def segment(img, out_path, id):

    '''
    Returns lobar segmentation numpy array of img and saves it to disk
    '''
        
    print('Segment lobes and save to', out_path)
    inferer = LMInferer(modelname='LTRCLobes', fillmodel='R231')
    segmentation_array = inferer.apply(img)
    segmentation_image = sitk.GetImageFromArray(segmentation_array)
    segmentation_image.CopyInformation(img)
    sitk.WriteImage(segmentation_image,
        os.path.join(out_path, f'{id}_segmentation.nii.gz'))

    return segmentation_array

## Batch process cases

In [ ]:
# List to store result dicts
results = []

# Dict to store failed cases
failed = {}

# Iterate through caseIDs
for c in cases:
    
    print('processing', c)

    # Dict to store per-case results
    res = {'caseID':c}

    try:
        
        # Load backdrop and iodine map
        backdrop = load(os.path.join(main_directory, c, 'backdrop'),
            os.path.join(main_directory, c), f'{c}_backdrop')

        iodine_map = load(os.path.join(main_directory, c, 'iodine_map'),
            os.path.join(main_directory, c), f'{c}_iodine_map')
        iodine_array = sitk.GetArrayFromImage(iodine_map)

        # Apply segmentation to backdrop
        segmentation_array = segment(backdrop, 
            os.path.join(main_directory, c), c)

        # Voxel spacing for volume calculation
        space = backdrop.GetSpacing()
        voxel = np.prod(space)

        # Volume of both lungs
        total_volume = voxel*np.sum(segmentation_array > 0)

        # Calculate mean iodine using segmentation array
        for lobe_name, lobe_number in lobes:
            
            perfusion_fraction = (np.sum(iodine_array[segmentation_array==lobe_number]) /
                np.sum(iodine_array[segmentation_array>0]))

            volume_fraction = (voxel*np.sum(segmentation_array == lobe_number) /
                total_volume)

            res[f'{lobe_name}_perfusion'] = np.round(perfusion_fraction, 2)
            res[f'{lobe_name}_volume'] = np.round(volume_fraction, 2)

        results.append(res)

    except Exception as e:
        print('could not process', c)
        print(str(e))
        failed[c] = e

# Save results to disk
results_df = pd.DataFrame(results)
results_df.to_excel(os.path.join(main_directory, 'results.xlsx'))

print('Failed cases:', failed)